# Prepare model files

In [1]:
import os
import glob
import json
from os.path import join as opj
import numpy as np
import pandas as pd

Project directories:

In [2]:
scratch_dir = os.environ['PI_SCRATCH']
data_dir = opj(scratch_dir, 'OPUS', 'BIDS_data')
model_dir = opj(data_dir, 'derivatives', 'l1model')

if not os.path.isdir(model_dir):
    os.mkdir(model_dir)
    
print('Reading data from: %s' % data_dir)
print('Placing model specification in: %s' % model_dir)

Reading data from: /scratch/groups/hyo/OPUS/BIDS_data
Placing model specification in: /scratch/groups/hyo/OPUS/BIDS_data/derivatives/l1model


Find subjects:

In [3]:
subjects = [os.path.basename(x) for x in glob.glob(opj(data_dir, 'sub-*'))]
print('Found subjects: %s' % subjects)

Found subjects: ['sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08']


## Self task

Contrasts:

In [4]:
self_names = ['self', 'motor-left', 'motor-right', 'self-chosen', 'self-unchosen']
self_con = [
    ('self: chosen > unchosen', 'T', self_names, [0, 0, 0, 1, -1]),
    ('self: unchosen > chosen', 'T', self_names, [0, 0, 0, -1, 1]),
    ('left > right', 'T', self_names, [0, 1, -1, 0, 0]),
    ('right > left', 'T', self_names, [0, -1, 1, 0, 0]),
    ('task', 'T', self_names, [1, 0, 0, 0, 0]),
    ('rest', 'T', self_names, [-1, 0, 0, 0, 0])
]

self_con_out = opj(model_dir, 'task-self_model-value.json')
with open(self_con_out, 'w') as fp:
    json.dump(self_con, fp, indent=4)

Make event files:

In [ ]:
for sub in subjects:
    # Create subject directory
    out_dir = opj(model_dir, sub, 'func')
    if not os.path.isdir(out_dir):
        os.makedirs(out_dir, exist_ok=True)
    print('Saving model specification to: %s' % out_dir)

    # Search for event files
    sub_dir = opj(data_dir, sub, 'func')
    event_files = glob.glob(opj(sub_dir, '*task-self*events.tsv'))
    print('Found functional runs: %s' % event_files)

    # Iterate over event files
    for event_f in event_files:
        events = pd.read_csv(event_f, sep='\t')
        
        # Main effect of trial
        trials = events.copy()[['onset', 'duration', 'trial_type']]

        # Motor response
        motor_response = pd.DataFrame()
        motor_response['onset'] = events['onset'] + events['response_time']
        motor_response['duration'] = 0.
        motor_response['trial_type'] = ['motor-%s' % (d) for d in np.where(events.choice == 1, 'left', 'right')]
        motor_response.dropna(inplace=True)

        # Chosen value
        chosen_value = trials.copy()
        chosen_value['trial_type'] = 'self-chosen'
        chosen_value['amplitude'] = events['self_value_c']
        chosen_value.dropna(inplace=True)

        # Unchosen value
        unchosen_value = trials.copy()
        unchosen_value['trial_type'] = 'self-unchosen'
        unchosen_value['amplitude'] = np.where(events['choice'] == 1, events['self_value2'], events['self_value1'])
        unchosen_value.dropna(inplace=True)

        # Build model
        model = pd.concat([trials, motor_response, chosen_value, unchosen_value])
        model = model[['onset', 'duration', 'trial_type', 'amplitude']]
        model.sort_values(axis=0, by=['onset', 'trial_type'], inplace=True)
        model.reset_index(drop=True, inplace=True)

        # Save output
        out_f = os.path.basename(event_f).replace('_run', '_model-value_run')
        out_path = opj(out_dir, out_f)
        model.to_csv(out_path, sep='\t', index=None)

## Test

Contrasts:

In [9]:
test_names = ['test', 'motor-left', 'motor-right', 
              'self-chosen', 'self-unchosen',
              'other-chosen', 'other-unchosen']
test_con = [
    ('self: chosen > unchosen', 'T', test_names, [0, 0, 0, 1, -1, 0, 0]),
    ('self: unchosen > chosen', 'T', test_names, [0, 0, 0, -1, 1, 0, 0]),
    ('other: chosen > unchosen', 'T', test_names, [0, 0, 0, 0, 0, 1, -1]),
    ('other: unchosen > chosen', 'T', test_names, [0, 0, 0, 0, 0, -1, 1]),
    ('left > right', 'T', test_names, [0, 1, -1, 0, 0, 0, 0]),
    ('right > left', 'T', test_names, [0, -1, 1, 0, 0, 0, 0]),
    ('task', 'T', test_names, [1, 0, 0, 0, 0, 0, 0]),
    ('rest', 'T', test_names, [-1, 0, 0, 0, 0, 0, 0])
]

test_con_out = opj(model_dir, 'task-test_model-value.json')
with open(test_con_out, 'w') as fp:
    json.dump(test_con, fp, indent=4)

Make event files:

In [ ]:
# for sub in subjects:
sub = subjects[0] # Debug

# Create subject directory
out_dir = opj(model_dir, sub, 'func')
if not os.path.isdir(out_dir):
    os.makedirs(out_dir, exist_ok=True)
print('Saving model specification to: %s' % out_dir)

# Search for event files
sub_dir = opj(data_dir, sub, 'func')
event_files = glob.glob(opj(sub_dir, '*task-test*events.tsv'))
print('Found functional runs: %s' % event_files)

# Iterate over event files
#for event_f in event_files:
event_f = event_files [0] # Debug
print('Reading data from: %s' % event_f)
events = pd.read_csv(event_f, sep='\t')

# Main effect of trial
trials = events.copy()[['onset', 'duration', 'trial_type']]

# Motor response
motor_response = pd.DataFrame()
motor_response['onset'] = events['onset'] + events['response_time']
motor_response['duration'] = 0.
motor_response['trial_type'] = ['motor-%s' % (d) for d in np.where(events.choice == 1, 'left', 'right')]
motor_response.dropna(inplace=True)

# Chosen value
chosen_value = trials.copy()
chosen_value['trial_type'] = 'self-chosen'
chosen_value['amplitude'] = events['self_value_c']
chosen_value.dropna(inplace=True)

# Unchosen value
unchosen_value = trials.copy()
unchosen_value['trial_type'] = 'self-unchosen'
unchosen_value['amplitude'] = np.where(events['choice'] == 1, events['self_value2'], events['self_value1'])
unchosen_value.dropna(inplace=True)

# Build model
model = pd.concat([trials, motor_response, chosen_value, unchosen_value])
model = model[['onset', 'duration', 'trial_type', 'amplitude']]
model.sort_values(axis=0, by=['onset', 'trial_type'], inplace=True)
model.reset_index(drop=True, inplace=True)

# # Save output
# out_f = os.path.basename(event_f).replace('_run', '_model-value_run')
# out_path = opj(out_dir, out_f)
# model.to_csv(out_path, sep='\t', index=None)

## Training

Contrasts:

Make event files:

## Probe

Contrasts:

Make event files: